In [7]:
import os
import ipynbname

chemin_notebook = ipynbname.path()
dossier_notebook = os.path.dirname(chemin_notebook)
os.chdir(dossier_notebook)
os.chdir('..')
print("Répertoire actuel :", os.getcwd())

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/My_project


In [8]:
from Utils.embedding import *

On va modifier l'embedding des types. On va réaliser un autoencodeur pour représenter la répart des types des pokes dans un espace 3 (et non 18). On garde un ordre dans ces représentations, car savoir quel type correspond à quel poké est important. Aussi on peut rajouter le nb_fainted, mais un vecteur 12 dim avec les positions des pokémons fainted, ça peut aider je pense à éviter les fausses actions.

In [9]:
#Instancier l'encodeur du type 
from Utils.autoencodeur.type.type_autoencodeur import TypeAutoencoder

encodeur_type = TypeAutoencoder(encoded_size=4)

# Charger les poids sauvegardés
encodeur_type.load_state_dict(torch.load("Utils/autoencodeur/type/type_autoencoder.pth", map_location='mps'))
encodeur_type.eval()

TypeAutoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=18, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=4, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=4, out_features=12, bias=True)
    (1): ReLU()
    (2): Linear(in_features=12, out_features=18, bias=True)
    (3): Sigmoid()
  )
)

Encodeur créé. Maintenant on remplace les types par les vecteurs encodés

In [10]:
############## DEFINIR L'EMBEDDING ##################
import numpy as np
from gymnasium.spaces import Space, Box
from poke_env.player import Gen8EnvSinglePlayer
from poke_env.data import GenData
import numpy as np
from poke_env.environment.abstract_battle import AbstractBattle
import torch
from gymnasium.spaces import Discrete, Box

# Initialiser GenData pour la génération souhaitée (par exemple, génération 8)
gen_data = GenData.from_gen(8)

# Accéder au tableau des types
type_chart = gen_data.type_chart


class embedding_Player(Gen8EnvSinglePlayer):
    def __init__(self,model_path=None, **kwargs):
        super().__init__(**kwargs)
        if model_path is not None :
            self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
            print(f"📥 Modèle chargé depuis {model_path}")

        self.action_space = Discrete(9)  # ✅ attribut classique
    
    #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        pokemon_types_compressed = []
        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 18)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )

        return np.float32(final_vector)

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 24 +         # my team types
            [0] * 24           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 24 +         # my team types
            [1] * 24           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action: int, battle: AbstractBattle):
        order = super().action_to_move(action, battle)
        order.dynamax = False  # 🔥 désactive Dynamax pour toutes les actions
        return order

In [11]:
########### DEFINIR MODELE ##################
import torch
import torch.nn as nn
import torch.nn.functional as F

#Réseau perceptron à une couche cachée, sortie linéaire, f activation = Relu,
class DQNModel(nn.Module):
    def __init__(self, input_dim, n_actions):
        super(DQNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=3, padding=1)
        self.conv12= nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.out = nn.Linear(32, n_actions)

    def forward(self, x):
        real_power = x[:4]
        types = x[4:] 
        real_power_padded = F.pad(real_power, (0, 14))  # [18]
        real_power_padded = real_power_padded.unsqueeze(0)  # [1, 18] 
        sequence = torch.cat([real_power_padded, types], dim=0)
        sequence = sequence.T.unsqueeze(0)     
        types = types.view(12, 18) 
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return self.out(x)

***TRAINING***

In [12]:

opponent = NoDynamaxRandomPlayer(battle_format="gen8randombattle")
from stable_baselines3.common.vec_env import DummyVecEnv

# instanciation du player
train_env_raw = embedding_Player(
    battle_format="gen8randombattle",
    opponent=RandomPlayer(battle_format="gen8randombattle"),
    start_challenging=True
)

# wrap dans DummyVecEnv (SB3 attend un vecteur d’envs, même pour un seul)
train_env = DummyVecEnv([lambda: train_env_raw])

In [13]:
env = train_env

model = DQN(
    policy="MlpPolicy",
    env=env,
    learning_rate=2.5e-4,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=32,
    gamma=0.5,
    train_freq=1,
    target_update_interval=1,
    exploration_fraction=1.0,
    exploration_final_eps=0.05,
    policy_kwargs=dict(activation_fn=nn.ReLU, net_arch=[128, 64])
)


In [14]:
#steps = 50000
#callback = CustomTQDMCallback(total_timesteps = steps, check_freq=1000, verbose=1)
#model.learn(
#    total_timesteps=steps,
#    callback=callback
#)
#model.save("Players/embedding_2")

In [15]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env

class EmbeddingTestPlayer(Gen8EnvSinglePlayer):

    def __init__(self, model_path="Players/embedding_2", **kwargs):
        super().__init__(**kwargs)
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print(f"📥 Modèle chargé depuis {model_path}")
        print("Répertoire actuel :", os.getcwd())

        #Toujours mêmes valeurs de reward
    def calc_reward(self, last_battle, current_battle) -> float:
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        pokemon_types_compressed = []
        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 18)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )

        return np.float32(final_vector)

    def describe_embedding(self) -> Space:
        low = (
            [-1] * 4 +          # real power
            [0] * 24 +         # my team types
            [0] * 24           # opponent team types
        )
        high = (
            [3] * 4 +           # real power
            [1] * 24 +         # my team types
            [1] * 24           # opponent team types
        )

        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32
        )
    
    def action_to_move(self, action, battle):
        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)

        #print(f"🔢 DQN → action={action} | #moves={len(moves)} | #switches={len(switches)} | total={total_actions}")

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            #print("❌ Action hors bornes ! Fallback sur move aléatoire")
            return self.choose_random_move(battle)
        
    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        q_values = self.model.q_net(obs_tensor)
        print(f"📊 Q-values : {q_values.detach().numpy().flatten()}")
        action = int(torch.argmax(q_values).item())
        return action

In [16]:
print("Répertoire actuel :", os.getcwd())
opponent = NoDynamaxRandomPlayer(battle_format="gen8randombattle")
eval_env = EmbeddingTestPlayer(
    battle_format="gen8randombattle", opponent=opponent, start_challenging=True
)

Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/My_project
📥 Modèle chargé depuis Players/embedding_2
Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/My_project


In [17]:
n_eval_episodes = 1
rewards = []
wins = 0

obs, _ = eval_env.reset()
for _ in tqdm(range(n_eval_episodes)):
    done = False
    total_reward = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, _ = eval_env.step(action)
        done = terminated or truncated

        total_reward += reward
        if done and reward > 0:
            wins += 1

    rewards.append(total_reward)
    obs, _ = eval_env.reset()

print(f"✅ {wins} victoires sur {n_eval_episodes} matchs")
print(f"🎯 Reward moyen : {sum(rewards) / len(rewards):.2f}")

  0%|          | 0/1 [00:00<?, ?it/s]

✅ 0 victoires sur 1 matchs
🎯 Reward moyen : -37.72


Adversarial Training

In [18]:
class AdversarialTrainPlayer(RandomPlayer):

    def __init__(self, model_path="Players/embedding_2", **kwargs):
        super().__init__(**kwargs)
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print(f"📥 Modèle chargé depuis {model_path}")

    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
                moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        pokemon_types_compressed = []
        #Pokemon types  
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 18)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )

        return np.float32(final_vector)
    
    #action_to_move suppr
        
    def predict(self, obs):
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        q_values = self.model.q_net(obs_tensor)
        print(f"📊 Q-values : {q_values.detach().numpy().flatten()}")
        action = int(torch.argmax(q_values).item())
        return action
    
    def choose_move(self, battle):
        obs = self.embed_battle(battle)
        device = "mps" if torch.backends.mps.is_available() else "cpu"
        obs_tensor = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
        with torch.no_grad():
            q_values = self.model.q_net(obs_tensor)
            action = int(torch.argmax(q_values).item())

        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            return self.choose_random_move(battle)

In [19]:

train_env_raw = embedding_Player(
    battle_format="gen8randombattle",
    opponent=AdversarialTrainPlayer(battle_format="gen8randombattle"),
    start_challenging=True
)
train_env = DummyVecEnv([lambda: train_env_raw])

#adv_learn_model = DQN.load("Players/embedding_2", env=train_env, device="mps" if torch.backends.mps.is_available() else "cpu")

# Puis on continue l'entraînement
#adv_learn_model.learn(total_timesteps=10000)

#adv_learn_model.save("Players/embedding_3")

📥 Modèle chargé depuis Players/embedding_2


In [20]:
opponent = AdversarialTrainPlayer(battle_format="gen8randombattle")

eval_env = EmbeddingTestPlayer(
    battle_format="gen8randombattle", opponent=opponent, start_challenging=True, model_path="Players/embedding_3"
)

📥 Modèle chargé depuis Players/embedding_2
📥 Modèle chargé depuis Players/embedding_3
Répertoire actuel : /Users/dan2/Desktop/Télécom-master-spé/Projets_perso/Deep/Showdown_AI/My_project


In [21]:
n_eval_episodes = 1
rewards = []
wins = 0

obs, _ = eval_env.reset()
for _ in tqdm(range(n_eval_episodes)):
    done = False
    total_reward = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, _ = eval_env.step(action)
        done = terminated or truncated

        total_reward += reward
        if done and reward > 0:
            wins += 1

    rewards.append(total_reward)
    obs, _ = eval_env.reset()

print(f"✅ {wins} victoires sur {n_eval_episodes} matchs")
print(f"🎯 Reward moyen : {sum(rewards) / len(rewards):.2f}")

  0%|          | 0/1 [00:00<?, ?it/s]

✅ 0 victoires sur 1 matchs
🎯 Reward moyen : -34.83


Je vais l'affronter moi 

In [22]:
from Utils.DQ_simple import DQ_simple 
class DQ_simpleemb3(DQ_simple) :
    def __init__(self, model_path = "Players/embedding_3", battle_format="gen8randombattle"):
        super().__init__(battle_format=battle_format)
        
        self.model = DQN.load(model_path, device="mps" if torch.backends.mps.is_available() else "cpu")
        print("📥 Modèle DQN chargé :", self.model)
    
    def embed_battle(self, battle )  :
        # -1 indicates that the move does not have a base power
        # or is not available
        moves_base_power = -np.ones(4)
        moves_dmg_multiplier = np.ones(4)
        moves_real_power = -np.ones(4)
        for i, move in enumerate(battle.available_moves):
            moves_base_power[i] = (
                move.base_power / 100
            )  # Simple rescaling to facilitate learning
            if move.type:
                moves_dmg_multiplier[i] = move.type.damage_multiplier(
                    battle.opponent_active_pokemon.type_1,
                    battle.opponent_active_pokemon.type_2,
                    type_chart=type_chart
                )
            moves_real_power[i] = moves_dmg_multiplier[i]*moves_base_power[i]

        #Pokemon types  
        pokemon_types_compressed = []
        pokemon_types = obtain_pokemon_types(battle)
        pokemon_types = torch.tensor(obtain_pokemon_types(battle)).view(12, 18)  # 6 pokés par team = 12 vecteurs
        with torch.no_grad():
            for i in range(12) :
                vec = pokemon_types[i].unsqueeze(0)
                encoded = encodeur_type.encoder(vec.float()) 
                pokemon_types_compressed.append(encoded.squeeze(0))

        # Final vector with 10 components
        compressed_flat = torch.cat(pokemon_types_compressed).numpy()
        final_vector = np.concatenate(
            [
                moves_real_power,
                compressed_flat,
            ]
        )

        return np.float32(final_vector)

    def choose_move(self, battle):
        print("👉 choose_move appelée !")
        # 🔍 Debug : Voir les moves disponibles
        print(f"🔍 Moves disponibles : {[move.id for move in battle.available_moves]}")

        # Obtenir l'observation de l'état du combat
        obs = self.embed_battle(battle)
        print("📊 Observation de l'état :", obs)

        # Transformer en format PyTorch
        obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
        print("📊 Tensor pour le modèle :", obs_tensor)

        # Prédire l'action avec le modèle DQN
        action = int(self.model.predict(obs_tensor, deterministic=True)[0])
        
        print("🎯 Action choisie par DQN :", action)
        moves = battle.available_moves
        switches = battle.available_switches
        total_actions = len(moves) + len(switches)
        self.create_order.dynamax = False

        if 0 <= action < len(moves):
            return self.create_order(moves[action])
        elif len(moves) <= action < total_actions:
            return self.create_order(switches[action - len(moves)])
        else:
            print("XXXX   : Random Moove ")
            return self.choose_random_move(battle)

In [23]:

bott_player = DQ_simpleemb3()

await bott_player.send_challenges("[NAME]", n_challenges=1)

📥 Modèle DQN chargé : <stable_baselines3.dqn.dqn.DQN object at 0x15ed63310>
📥 Modèle DQN chargé : <stable_baselines3.dqn.dqn.DQN object at 0x15ed6d9d0>
👉 choose_move appelée !
🔍 Moves disponibles : ['sleeppowder', 'airslash', 'bugbuzz', 'quiverdance']
📊 Observation de l'état : [ 0.          1.5         0.45        0.         -1.7605909   0.6749176
 -0.845018    0.83518666 -1.544438    1.5230358  -1.0729465   0.8510073
 -1.0553175   1.4143128  -0.56194496  0.49795628 -1.9057633   0.99372894
 -1.1991675   0.9391059  -1.8650103   1.3752306  -1.1475861   0.9287637
 -1.5516384   0.791832   -0.8793163   0.74129087 -1.6891162   1.1604713
 -0.8384739   0.86872107 -1.152232    1.0118549  -0.49244103  0.46972787
 -1.152232    1.0118549  -0.49244103  0.46972787 -1.152232    1.0118549
 -0.49244103  0.46972787 -1.152232    1.0118549  -0.49244103  0.46972787
 -1.152232    1.0118549  -0.49244103  0.46972787]
📊 Tensor pour le modèle : tensor([[ 0.0000,  1.5000,  0.4500,  0.0000, -1.7606,  0.6749, -0.8

/var/folders/h3/sk18hvh55gqbml8209nmkrlc0000gq/T/ipykernel_31401/204959055.py:62: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  action = int(self.model.predict(obs_tensor, deterministic=True)[0])


👉 choose_move appelée !
🔍 Moves disponibles : ['sleeppowder', 'airslash', 'bugbuzz', 'quiverdance']
📊 Observation de l'état : [ 0.          0.375       0.9         0.         -1.7605909   0.6749176
 -0.845018    0.83518666 -1.544438    1.5230358  -1.0729465   0.8510073
 -1.0553175   1.4143128  -0.56194496  0.49795628 -1.9057633   0.99372894
 -1.1991675   0.9391059  -1.8650103   1.3752306  -1.1475861   0.9287637
 -1.5516384   0.791832   -0.8793163   0.74129087 -1.152232    1.0118549
 -0.49244103  0.46972787 -1.1690445   1.6854677  -0.6401736   0.5895755
 -1.152232    1.0118549  -0.49244103  0.46972787 -1.152232    1.0118549
 -0.49244103  0.46972787 -1.152232    1.0118549  -0.49244103  0.46972787
 -1.152232    1.0118549  -0.49244103  0.46972787]
📊 Tensor pour le modèle : tensor([[ 0.0000,  0.3750,  0.9000,  0.0000, -1.7606,  0.6749, -0.8450,  0.8352,
         -1.5444,  1.5230, -1.0729,  0.8510, -1.0553,  1.4143, -0.5619,  0.4980,
         -1.9058,  0.9937, -1.1992,  0.9391, -1.8650,  1.3

2025-03-31 01:42:50,919 - DQ_simpleemb3 3 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Gyarados', 'Bounce', 'p1a: Wishiwashi', '[from] lockedmove'] in battle battle-gen8randombattle-6500 turn 12


👉 choose_move appelée !
🔍 Moves disponibles : ['uturn', 'scald', 'icebeam', 'earthquake']
📊 Observation de l'état : [ 0.35        0.4         0.9         0.         -1.7605909   0.6749176
 -0.845018    0.83518666 -1.544438    1.5230358  -1.0729465   0.8510073
 -1.152232    1.0118549  -0.49244103  0.46972787 -1.152232    1.0118549
 -0.49244103  0.46972787 -1.8650103   1.3752306  -1.1475861   0.9287637
 -1.5516384   0.791832   -0.8793163   0.74129087 -1.152232    1.0118549
 -0.49244103  0.46972787 -1.152232    1.0118549  -0.49244103  0.46972787
 -2.129465    0.48220807 -1.1449594   0.99173605 -1.152232    1.0118549
 -0.49244103  0.46972787 -1.152232    1.0118549  -0.49244103  0.46972787
 -1.152232    1.0118549  -0.49244103  0.46972787]
📊 Tensor pour le modèle : tensor([[ 0.3500,  0.4000,  0.9000,  0.0000, -1.7606,  0.6749, -0.8450,  0.8352,
         -1.5444,  1.5230, -1.0729,  0.8510, -1.1522,  1.0119, -0.4924,  0.4697,
         -1.1522,  1.0119, -0.4924,  0.4697, -1.8650,  1.3752, -1.14

2025-03-31 01:42:55,409 - DQ_simpleemb3 3 - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p2a: Gyarados', 'Bounce', 'p1a: Wishiwashi', '[from] lockedmove'] in battle battle-gen8randombattle-6500 turn 14


👉 choose_move appelée !
🔍 Moves disponibles : ['uturn', 'scald', 'icebeam', 'earthquake']
📊 Observation de l'état : [ 0.35        0.4         0.45        1.         -1.7605909   0.6749176
 -0.845018    0.83518666 -1.544438    1.5230358  -1.0729465   0.8510073
 -1.152232    1.0118549  -0.49244103  0.46972787 -1.152232    1.0118549
 -0.49244103  0.46972787 -1.8650103   1.3752306  -1.1475861   0.9287637
 -1.5516384   0.791832   -0.8793163   0.74129087 -1.152232    1.0118549
 -0.49244103  0.46972787 -1.152232    1.0118549  -0.49244103  0.46972787
 -1.152232    1.0118549  -0.49244103  0.46972787 -1.7280061   1.0265892
 -1.2733984   0.9492904  -1.152232    1.0118549  -0.49244103  0.46972787
 -1.152232    1.0118549  -0.49244103  0.46972787]
📊 Tensor pour le modèle : tensor([[ 0.3500,  0.4000,  0.4500,  1.0000, -1.7606,  0.6749, -0.8450,  0.8352,
         -1.5444,  1.5230, -1.0729,  0.8510, -1.1522,  1.0119, -0.4924,  0.4697,
         -1.1522,  1.0119, -0.4924,  0.4697, -1.8650,  1.3752, -1.14